In [1]:
#!/usr/bin/python


<br>
Estimate 2D free energy surface for alanine dipeptide parallel tempering data using MBAR.<br>
PROTOCOL<br>
* Potential energies and (phi, psi) torsions from parallel tempering simulation are read in by temperature<br>
* Replica trajectories of potential energies and torsions are reconstructed to reflect their true temporal<br>
  correlation, and then subsampled to produce statistically independent samples, collecting them again by temperature<br>
* The `pymbar` class is initialized to compute the dimensionless free energies at each temperature using MBAR<br>
* The torsions are binned into sequentially labeled bins in two dimensions<br>
* The relative free energies and uncertainties of these torsion bins at the temperature of interest is estimated<br>
* The 2D free energy surface is written out<br>
REFERENCES<br>
[1] Shirts MR and Chodera JD. Statistically optimal analysis of samples from multiple equilibrium states.<br>
J. Chem. Phys. 129:124105, 2008. http://dx.doi.org/10.1063/1.2978177<br>


===================================================================================================<br>
IMPORTS<br>
===================================================================================================

In [2]:
from pathlib import Path

In [3]:
import numpy as np
import pymbar  # for MBAR analysis

Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.

****** PyMBAR will use 64-bit JAX! *******
* JAX is currently set to 32-bit bitsize *
* which is its default.                  *
*                                        *
* PyMBAR requires 64-bit mode and WILL   *
* enable JAX's 64-bit mode when called.  *
*                                        *
* This MAY cause problems with other     *
* Uses of JAX in the same code.          *
******************************************



In [4]:
from pymbar import timeseries  # for timeseries analysis

===================================================================================================<br>
CONSTANTS<br>
===================================================================================================

In [5]:
kB = 1.3806503 * 6.0221415 / 4184.0  # Boltzmann constant in kcal/mol/K

===================================================================================================<br>
PARAMETERS<br>
===================================================================================================

In [6]:
DATA_DIRECTORY = Path("data/")  # directory containing the parallel tempering data
# file containing temperatures in K
temperature_list_filename = DATA_DIRECTORY / "temperatures"
free_energies_filename = "f_k.out"
# file containing total energies (in kcal/mol) for each temperature and snapshot
potential_energies_filename = DATA_DIRECTORY / "energies" / "potential-energies"
trajectory_segment_length = 20  # number of snapshots in each contiguous trajectory segment
niterations = 500  # number of iterations to use
target_temperature = 302  # target temperature for 2D free energy surface (in K)
nbins_per_torsion = 10  # number of bins per torsion dimension

===================================================================================================<br>
SUBROUTINES<br>
===================================================================================================

In [7]:
def read_file(filename):
    """Read contents of the specified file.
    Parameters:
    -----------
    filename : str
        The name of the file to be read
    Returns
    -------
    lines : list of str
        The contents of the file, split by line
    """
    with open(filename, "r") as f:
        return f.readlines()

===================================================================================================<br>
MAIN<br>
===================================================================================================

===================================================================================================<br>
Read temperatures<br>
===================================================================================================

Read list of temperatures.

In [8]:
lines = read_file(temperature_list_filename)
# Construct list of temperatures
temperatures = lines[0].split()
# Create array of temperatures
K = len(temperatures)
temperature_k = np.zeros([K])  # temperature_k[k] is temperature of temperature index k in K
for k in range(K):
    temperature_k[k] = float(temperatures[k])
# Compute inverse temperatures
beta_k = (kB * temperature_k) ** (-1)

Define other constants

In [9]:
T = trajectory_segment_length * niterations  # total number of snapshots per temperature

===================================================================================================<br>
Read potential eneriges<br>
===================================================================================================

In [10]:
print("Reading potential energies...")
# U_kn[k,t] is the potential energy (in kcal/mol) for snapshot t of temperature index k
U_kt = np.zeros([K, T])
lines = read_file(potential_energies_filename)
print(f"{len(lines):d} lines read, processing {T:d} snapshots")
for t in range(T):
    # Get line containing the energies for snapshot t of trajectory segment n
    line = lines[t]
    # Extract energy values from text
    elements = line.split()
    for k in range(K):
        U_kt[k, t] = float(elements[k])

Reading potential energies...
10000 lines read, processing 10000 snapshots


===================================================================================================<br>
Read phi, psi trajectories<br>
===================================================================================================

In [11]:
print("Reading phi, psi trajectories...")
# phi_kt[k,n,t] is phi angle (in degrees) for snapshot t of temperature k
phi_kt = np.zeros([K, T])
# psi_kt[k,n,t] is psi angle (in degrees) for snapshot t of temperature k
psi_kt = np.zeros([K, T])
for k in range(K):
    phi_filename = DATA_DIRECTORY / "backbone-torsions" / f"{k:d}.phi"
    psi_filename = DATA_DIRECTORY / "backbone-torsions" / f"{k:d}.psi"
    phi_lines = read_file(phi_filename)
    psi_lines = read_file(psi_filename)
    print(f"k = {k:d}, {len(phi_lines):d} phi lines read, {len(psi_lines):d} psi lines read")
    for t in range(T):
        # Extract phi and psi
        phi_kt[k, t] = float(phi_lines[t])
        psi_kt[k, t] = float(psi_lines[t])

Reading phi, psi trajectories...
k = 0, 10000 phi lines read, 10000 psi lines read
k = 1, 10000 phi lines read, 10000 psi lines read
k = 2, 10000 phi lines read, 10000 psi lines read
k = 3, 10000 phi lines read, 10000 psi lines read
k = 4, 10000 phi lines read, 10000 psi lines read
k = 5, 10000 phi lines read, 10000 psi lines read
k = 6, 10000 phi lines read, 10000 psi lines read
k = 7, 10000 phi lines read, 10000 psi lines read
k = 8, 10000 phi lines read, 10000 psi lines read
k = 9, 10000 phi lines read, 10000 psi lines read
k = 10, 10000 phi lines read, 10000 psi lines read
k = 11, 10000 phi lines read, 10000 psi lines read
k = 12, 10000 phi lines read, 10000 psi lines read
k = 13, 10000 phi lines read, 10000 psi lines read
k = 14, 10000 phi lines read, 10000 psi lines read
k = 15, 10000 phi lines read, 10000 psi lines read
k = 16, 10000 phi lines read, 10000 psi lines read
k = 17, 10000 phi lines read, 10000 psi lines read
k = 18, 10000 phi lines read, 10000 psi lines read
k = 19, 

===================================================================================================<br>
Read replica indices<br>
===================================================================================================

In [12]:
print("Reading replica indices...")
filename = DATA_DIRECTORY / "replica-indices"
lines = read_file(filename)
# replica_ki[i,k] is the replica index of temperature k for iteration i
replica_ik = np.zeros([niterations, K], np.int32)
for i in range(niterations):
    elements = lines[i].split()
    for k in range(K):
        replica_ik[i, k] = int(elements[k])
print(f"Replica indices for {niterations:d} iterations processed.")

Reading replica indices...
Replica indices for 500 iterations processed.


===================================================================================================<br>
Permute data by replica and subsample to generate an uncorrelated subset of data by temperature<br>
===================================================================================================

In [13]:
assume_uncorrelated = False
if assume_uncorrelated:
    # DEBUG - use all data, assuming it is uncorrelated
    print("Using all data, assuming it is uncorrelated...")
    U_kn = U_kt.copy()
    phi_kn = phi_kt.copy()
    psi_kn = psi_kt.copy()
    N_k = np.zeros([K], np.int32)
    N_k[:] = T
    N_max = T
else:
    # Permute data by replica
    print("Permuting data by replica...")
    U_kt_replica = U_kt.copy()
    phi_kt_replica = psi_kt.copy()
    psi_kt_replica = psi_kt.copy()
    for iteration in range(niterations):
        # Determine which snapshot indices are associated with this iteration
        snapshot_indices = iteration * trajectory_segment_length + np.arange(
            0, trajectory_segment_length
        )
        for k in range(K):
            # Determine which replica generated the data from temperature k at this iteration
            replica_index = replica_ik[iteration, k]
            # Reconstruct portion of replica trajectory.
            U_kt_replica[replica_index, snapshot_indices] = U_kt[k, snapshot_indices]
            phi_kt_replica[replica_index, snapshot_indices] = phi_kt[k, snapshot_indices]
            psi_kt_replica[replica_index, snapshot_indices] = psi_kt[k, snapshot_indices]
    # Estimate the statistical inefficiency of the simulation by analyzing the timeseries of interest.
    # We use the max of cos and sin of the phi and psi timeseries because they are periodic angles.
    # The  ## TODO: ???
    print("Computing statistical inefficiencies...")
    g_cosphi = timeseries.statistical_inefficiency_multiple(np.cos(phi_kt_replica * np.pi / 180.0))
    print(f"g_cos(phi) = {g_cosphi:.1f}")
    g_sinphi = timeseries.statistical_inefficiency_multiple(np.sin(phi_kt_replica * np.pi / 180.0))
    print(f"g_sin(phi) = {g_sinphi:.1f}")
    g_cospsi = timeseries.statistical_inefficiency_multiple(np.cos(psi_kt_replica * np.pi / 180.0))
    print(f"g_cos(psi) = {g_cospsi:.1f}")
    g_sinpsi = timeseries.statistical_inefficiency_multiple(np.sin(psi_kt_replica * np.pi / 180.0))
    print(f"g_sin(psi) = {g_sinpsi:.1f}")
    # Subsample data with maximum of all correlation times.
    print("Subsampling data...")
    g = np.max(np.array([g_cosphi, g_sinphi, g_cospsi, g_sinpsi]))
    indices = timeseries.subsample_correlated_data(U_kt[k, :], g=g)
    print(f"Using g = {g:.1f} to obtain {len(indices):d} uncorrelated samples per temperature")
    N_max = int(np.ceil(T / g))  # max number of samples per temperature
    U_kn = np.zeros([K, N_max])
    phi_kn = np.zeros([K, N_max])
    psi_kn = np.zeros([K, N_max])
    N_k = N_max * np.ones([K], dtype=int)
    for k in range(K):
        U_kn[k, :] = U_kt[k, indices]
        phi_kn[k, :] = phi_kt[k, indices]
        psi_kn[k, :] = psi_kt[k, indices]
    print(f"{N_max:d} uncorrelated samples per temperature")

Permuting data by replica...
Computing statistical inefficiencies...
g_cos(phi) = 5.6
g_sin(phi) = 11.2
g_cos(psi) = 149.7
g_sin(psi) = 49.3
Subsampling data...
Using g = 149.7 to obtain 67 uncorrelated samples per temperature
67 uncorrelated samples per temperature


===================================================================================================<br>
Generate a list of indices of all configurations in kn-indexing<br>
===================================================================================================

Create a list of indices of all configurations in kn-indexing.

In [14]:
mask_kn = np.zeros([K, N_max], dtype=bool)
for k in range(K):
    mask_kn[k, 0 : N_k[k]] = True
# Create a list from this mask.
indices = np.where(mask_kn)

===================================================================================================<br>
Compute reduced potential energy of all snapshots at all temperatures<br>
===================================================================================================

In [15]:
print("Computing reduced potential energies...")
# u_kln[k,l,n] is reduced potential energy of trajectory segment n of temperature k evaluated at temperature l
u_kln = np.zeros([K, K, N_max])
for k in range(K):
    for l in range(K):
        u_kln[k, l, 0 : N_k[k]] = beta_k[l] * U_kn[k, 0 : N_k[k]]

Computing reduced potential energies...


===================================================================================================<br>
Bin torsions into histogram bins for free energy surface calculation<br>
===================================================================================================

Here, we bin the (phi,psi) samples into bins in a 2D histogram.<br>
We assign indices 0...(nbins-1) to the bins, even though the histograms are in two dimensions.<br>
All bins must have at least one sample in them.<br>
This strategy scales to an arbitrary number of dimensions.

In [16]:
print("Binning torsions...")
# Determine torsion bin size (in degrees)
torsion_min = -180.0
torsion_max = +180.0
dx = (torsion_max - torsion_min) / float(nbins_per_torsion)
# Assign torsion bins
# bin_kn[k,n] is the index of which histogram bin sample n from temperature index k belongs to
bin_kn = np.zeros([K, N_max], dtype=int)
nbins = 0
bin_nonzero = 0
bin_counts = []
bin_centers = []  # bin_centers[i] is a (phi,psi) tuple that gives the center of bin i
count_nonzero = []
centers_nonzero = []

Binning torsions...


In [17]:
for i in range(nbins_per_torsion):
    for j in range(nbins_per_torsion):
        # Determine (phi,psi) of bin center.
        phi = torsion_min + dx * (i + 0.5)
        psi = torsion_min + dx * (j + 0.5)

        # Determine which configurations lie in this bin.
        in_bin = (
            (phi - dx / 2 <= phi_kn[indices])
            & (phi_kn[indices] < phi + dx / 2)
            & (psi - dx / 2 <= psi_kn[indices])
            & (psi_kn[indices] < psi + dx / 2)
        )
        # Count number of configurations in this bin.
        bin_count = in_bin.sum()
        # Generate list of indices in bin.
        # set bin indices of both dimensions
        if bin_count > 0:
            count_nonzero.append(bin_count)
            centers_nonzero.append((phi, psi))
            bin_nonzero += 1

In [18]:
print(f"{bin_nonzero:d} bins were populated:")
for i in range(bin_nonzero):
    print(
        f"bin {i:5d} ({centers_nonzero[i][0]:6.1f}, {centers_nonzero[i][1]:6.1f}) {count_nonzero[i]:12d} conformations"
    )

67 bins were populated:
bin     0 (-162.0, -162.0)           73 conformations
bin     1 (-162.0, -126.0)           14 conformations
bin     2 (-162.0,  -90.0)           14 conformations
bin     3 (-162.0,  -54.0)           27 conformations
bin     4 (-162.0,  -18.0)            3 conformations
bin     5 (-162.0,   18.0)            8 conformations
bin     6 (-162.0,   54.0)           18 conformations
bin     7 (-162.0,   90.0)           42 conformations
bin     8 (-162.0,  126.0)          141 conformations
bin     9 (-162.0,  162.0)          262 conformations
bin    10 (-126.0, -162.0)           99 conformations
bin    11 (-126.0, -126.0)           20 conformations
bin    12 (-126.0,  -90.0)           32 conformations
bin    13 (-126.0,  -54.0)           51 conformations
bin    14 (-126.0,  -18.0)           20 conformations
bin    15 (-126.0,   18.0)           18 conformations
bin    16 (-126.0,   54.0)           15 conformations
bin    17 (-126.0,   90.0)           46 conformations
bin 

In [19]:
x_n = np.zeros([np.sum(N_k), 2])  # the configurations

In [20]:
Ntot = 0
for k in range(K):
    for n in range(N_k[k]):
        x_n[Ntot, 0] = phi_kn[k, n]
        x_n[Ntot, 1] = psi_kn[k, n]
        Ntot += 1

In [21]:
bin_edges = []
for i in range(2):
    bin_edges.append(np.linspace(torsion_min, torsion_max, nbins_per_torsion + 1))

Initialize free energy surface with data collected

In [22]:
fes = pymbar.FES(u_kln, N_k, mbar_options=dict(solver_protocol="robust"))


******* JAX 64-bit mode is now on! *******
*     JAX is now set to 64-bit mode!     *
*   This MAY cause problems with other   *
*      uses of JAX in the same code.     *
******************************************



===================================================================================================<br>
Compute free energy surface at the desired temperature.<br>
===================================================================================================

In [23]:
print("Computing free energy surface...")

Computing free energy surface...


Compute reduced potential energies at the temperaure of interest

In [24]:
target_beta = 1.0 / (kB * target_temperature)
u_kn = target_beta * U_kn
# Compute FES at this temperature, returning dimensionless free energies and uncertainties.
# f_i[i] is the dimensionless free energy of bin i (in kT) at the temperature of interest
# df_i[i,j] is an estimate of the covariance in the estimate of (f_i[i] - f_j[j], with reference
# the lowest free energy state.
# Compute FES in unbiased potential (in units of kT).
histogram_parameters = {}
histogram_parameters["bin_edges"] = bin_edges
fes.generate_fes(u_kn, x_n, fes_type="histogram", histogram_parameters=histogram_parameters)
results = fes.get_fes(
    np.array(centers_nonzero), reference_point="from-lowest", uncertainty_method="analytical"
)
f_i = results["f_i"]
df_i = results["df_i"]

Show free energy and uncertainty of each occupied bin relative to lowest free energy

In [25]:
print("2D free energy surface")
print()
print(f"{'bin':>8s} {'phi':>6s} {'psi':>6s} {'N':>8s} {'f':>10s} {'df':>10s}")

2D free energy surface

     bin    phi    psi        N          f         df


In [26]:
for i in range(bin_nonzero):
    print(
        f"{i:>8d} {centers_nonzero[i][0]:>6.1f} {centers_nonzero[i][1]:>6.1f} {count_nonzero[i]:>8d} {f_i[i]:>10.3f} {df_i[i]:>10.3f}"
    )

       0 -162.0 -162.0       73      1.977      0.459
       1 -162.0 -126.0       14      4.928      1.018
       2 -162.0  -90.0       14      2.976      0.775
       3 -162.0  -54.0       27      3.620      0.703
       4 -162.0  -18.0        3      5.846      1.000
       5 -162.0   18.0        8      4.586      1.018
       6 -162.0   54.0       18     33.383      1.057
       7 -162.0   90.0       42      2.626      0.656
       8 -162.0  126.0      141      1.261      0.361
       9 -162.0  162.0      262      0.468      0.268
      10 -126.0 -162.0       99      1.511      0.369
      11 -126.0 -126.0       20      2.865      0.652
      12 -126.0  -90.0       32      3.501      1.004
      13 -126.0  -54.0       51      2.529      0.672
      14 -126.0  -18.0       20      3.564      0.877
      15 -126.0   18.0       18      3.759      0.893
      16 -126.0   54.0       15      3.201      0.755
      17 -126.0   90.0       46      2.406      0.565
      18 -126.0  126.0      